In [8]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

model = YOLO('yolov8s.pt')
tracker = DeepSort(max_age=10)

cap = cv2.VideoCapture('SeoulWalk2_720.mp4')
track_paths = {}
color_map = {}

# ==========바운딩박스 교차 비교 확인 함수 =========
def is_intersect(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    return not (
        x1_max < x2_min or
        x2_max < x1_min or
        y1_max < y2_min or
        y2_max < y1_min
    )
# =============================================

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    results = model(frame, verbose=False)[0]
    #frame = results.plot()

    detections = []
    
    # =======영상 프레임 크기 기준 사용자 근접 영역 계산 =========
    frame_height, frame_width = frame.shape[:2]
    user_x = frame_width//2
    user_y = frame_height - 10
    near_left = frame_width //4
    near_right = frame_width * 3 //4
    near_top = frame_height *3 //4
    user_area = (near_left, near_top, near_right, frame_height)
    # =====================================================
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))

    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = (x1 + x2) // 2, (y1 + y2) //2
        
        if track_id not in color_map:
            color_map[track_id] = tuple(np.random.randint(0, 255, size=3).tolist())
        color = color_map[track_id]
        
        class_name = track.get_det_class()

        # 객체 박스 좌표를 변수에 저장하고, 사용자 근접 영역과 겹치는지 확인 -> 겹치는 객체만 박스 표시
        object_box = (x1, y1, x2, y2)
        if is_intersect(object_box, user_area):                    
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'ID {track_id} {class_name}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        # =============================================================================================
        
        if track_id not in track_paths:
            track_paths[track_id] = []
        track_paths[track_id].append((cx, cy))
        
        # 객체 이동 경로를 선으로 표시하는 코드
        path = track_paths[track_id]
        for i in range(1, len(path)):
            cv2.line(frame, path[i-1], path[i], color, 2)

    # 사용자를 작은 원으로 표시하고 사용자 근접 영역을 박스로 표시
    cv2.circle(frame, (user_x, user_y), 10, (0,0,255), -1)
    cv2.putText(frame, 'User', (user_x-10, user_y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),2)
    cv2.rectangle(frame, (near_left, near_top), (near_right, frame_height), (255,0,0), 2)
    
    cv2.imshow("YOLOv8 plot + Deep Sort Tracking Visualization", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()